In [1]:
import numpy as np
import imageio
from hrl.utils import predict_ball_analytic
import mujoco

from myosuite.utils import gym

MyoSuite:> Registering Myo Envs


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment myoArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment myoSarcArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {ne

In [6]:
import os, inspect, xml.etree.ElementTree as ET
import myosuite.envs.myo.myochallenge as mc

# Find assets dir
pkg_path = os.path.dirname(inspect.getfile(mc))
assets_dir = "/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/myosuite/envs/myo/assets"

# Parse track XML
track_xml = os.path.join(assets_dir, "myosuite_track.xml")
root = ET.parse(track_xml).getroot()

# Gravity
for opt in root.findall("option"):
    print("Gravity:", opt.attrib.get("gravity"))

# Table geoms
for geom in root.findall(".//geom"):
    name = geom.attrib.get("name", "")
    mesh = geom.attrib.get("mesh", "")
    if "table" in name.lower() or "table" in mesh.lower():
        print("Table geom:", geom.attrib)

# Parse paddle XML
paddle_xml = os.path.join(assets_dir, "paddle.xml")
root2 = ET.parse(paddle_xml).getroot()
for geom in root2.findall(".//geom"):
    print("Paddle geom:", geom.attrib)
    

Paddle geom: {'name': 'ping_pong_paddle', 'type': 'mesh', 'mesh': 'ping_pong_paddle_mesh', 'material': 'paddle_mat', 'rgba': '1 1 1 1', 'pos': '0 0 0', 'euler': '0 0 0'}


In [14]:
for step in range(200):
    action = env.action_space.sample()

    obs, base_reward, terminated, truncated, info = env.step(action)

    # ---------------------------------
    # Extract observations (YOUR WAY)
    # ---------------------------------
    obs_dict = info["obs_dict"]
    ball_xyz = obs_dict["ball_pos"]
    ball_vel = obs_dict["ball_vel"]
    paddle_xyz = obs_dict["paddle_pos"]

    print(f"[{step:03d}] ball_xyz={ball_xyz}, ball_vel={ball_vel}")

    # ---------------------------------
    # Call YOUR fixed predictor
    # ---------------------------------
    pred_pos, n_ideal, quat_ideal = predict_ball_analytic(
        sim=env.unwrapped.sim,
        id_info=env.unwrapped.id_info,
        ball_pos=ball_xyz,
        ball_vel=ball_vel,
        paddle_pos=paddle_xyz,
    )

    print(
        f"    pred_hit={pred_pos}, "
        f"n_ideal={n_ideal}, "
        f"dot(+X,n)={n_ideal[0]:.3f}"
    )

    # ---------------------------------
    # OFFSCREEN RENDER (ADDED)
    # ---------------------------------
    frame = renderer.render_offscreen(
        width=width,
        height=height,
        camera_id=1,   # default camera
    )
    writer.append_data(frame)

    if terminated or truncated:
        break

# -----------------------------------------
# Cleanup
# -----------------------------------------
writer.close()
env.close()

print("Done. Video saved:", video_path)

[000] ball_xyz=[-0.9557  0.2276  1.4652], ball_vel=[ 4.3013  0.9219 -0.1693]
    pred_hit=[1.91   0.8418 1.3425], n_ideal=[0.9033 0.3852 0.1889], dot(+X,n)=0.903
[001] ball_xyz=[-0.9126  0.2368  1.463 ], ball_vel=[ 4.3013  0.9219 -0.2674]
    pred_hit=[1.9101 0.8418 1.3424], n_ideal=[0.9033 0.3852 0.1888], dot(+X,n)=0.903
[002] ball_xyz=[-0.8696  0.246   1.4597], ball_vel=[ 4.3013  0.9219 -0.3655]
    pred_hit=[1.9102 0.8418 1.3423], n_ideal=[0.9033 0.3852 0.1887], dot(+X,n)=0.903
[003] ball_xyz=[-0.8266  0.2552  1.4555], ball_vel=[ 4.3013  0.9219 -0.4636]
    pred_hit=[1.9089 0.8415 1.3422], n_ideal=[0.9032 0.3853 0.1891], dot(+X,n)=0.903
[004] ball_xyz=[-0.7836  0.2644  1.4502], ball_vel=[ 4.3013  0.9219 -0.5617]
    pred_hit=[1.9071 0.8411 1.3421], n_ideal=[0.903  0.3855 0.1897], dot(+X,n)=0.903
[005] ball_xyz=[-0.7406  0.2737  1.444 ], ball_vel=[ 4.3013  0.9219 -0.6598]
    pred_hit=[1.9013 0.8399 1.3419], n_ideal=[0.9024 0.386  0.1917], dot(+X,n)=0.902
[006] ball_xyz=[-0.6976  0.2